# TASK: 1 - Data Quality Assessment
### Assessment of data quality and completeness in preparation for analysis.
(**Transactions data in the past 3 months**)

In [11]:
# importing required libraries 
import pandas as pd
import numpy as np
import datetime as dt

### Reading data

In [12]:
import os
file_name = 'KPMG_VI_New_raw_data_update_final.xlsx'

# Get the current working directory
current_directory = os.getcwd()

# Create the complete file path by joining the directory and file name
file_path = os.path.join(current_directory, file_name)
# importing data from "Transactions" sheet
transactions=pd.read_excel(file_path,sheet_name="Transactions")

### Exploring transaction data

In [13]:
transactions.head(5)

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
0,1,2,2950,2017-02-25,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,41245.0
1,2,3,3120,2017-05-21,1.0,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,41701.0
2,3,37,402,2017-10-16,0.0,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,36361.0
3,4,88,3135,2017-08-31,0.0,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,36145.0
4,5,78,787,2017-10-01,1.0,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,42226.0


In [14]:
# checking the data
print("The shape of data->",transactions.shape)
print()
transactions.info()

The shape of data-> (20000, 13)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   transaction_id           20000 non-null  int64         
 1   product_id               20000 non-null  int64         
 2   customer_id              20000 non-null  int64         
 3   transaction_date         20000 non-null  datetime64[ns]
 4   online_order             19640 non-null  float64       
 5   order_status             20000 non-null  object        
 6   brand                    19803 non-null  object        
 7   product_line             19803 non-null  object        
 8   product_class            19803 non-null  object        
 9   product_size             19803 non-null  object        
 10  list_price               20000 non-null  float64       
 11  standard_cost            19803 non-null  float64       
 12 

#### Initially there is 20000 rows and 13 coloumn

In [15]:
# checking the null values
transactions.isnull().sum()

transaction_id               0
product_id                   0
customer_id                  0
transaction_date             0
online_order               360
order_status                 0
brand                      197
product_line               197
product_class              197
product_size               197
list_price                   0
standard_cost              197
product_first_sold_date    197
dtype: int64

#### There is null values in 7 coloumns.We can drop them or replace them accroding to the nature of analysis or data trend.
*Such as we can replace the null values of the "standard_cost" column as per the overall standard cost of this particular product_id , so we can raplace them with the mean of standard_cost of particular product_id.*

*Although we prefer to replace the values of numerical column only , there no need to replacs the null values of categorical & identical column.*

#### We can  change all data value into one case 

In [16]:
def change_upper(df):
    for column in df.columns:
        if df[column].dtype=='object':
            df[column] = df[column].str.upper()
    return df

#### we can use change_upper fuction changing the data values to upper case
*so there is consistency in the records*

#### Checking the duplicate values

In [17]:
transactions.duplicated().sum()

0

#### There is no duplicate records present in the data set.
*so all records are unique.*

### Exploring coloumns

#### Generate a summary of count of all the outliers column wise

In [18]:
def detect_outliers_iqr(dataframe,column_name):
    # Calculate quartiles
    Q1 = dataframe[column_name].quantile(0.25)
    Q3 = dataframe[column_name].quantile(0.75)
    
    # Calculate IQR
    IQR = Q3 - Q1
    
    # Calculate lower and upper bounds
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Find outliers
    outliers = dataframe[(dataframe[column_name] < lower_bound) | (dataframe[column_name] > upper_bound)]
    
    return outliers

In [19]:
for column in transactions:
    if transactions[column].dtype=='int64' or transactions[column].dtype=='float64':
        outliers=detect_outliers_iqr(transactions,column)
        print(f"the no. of outlier in {column} is:")
        print(outliers[column].count())

the no. of outlier in transaction_id is:
0
the no. of outlier in product_id is:
0
the no. of outlier in customer_id is:
0
the no. of outlier in online_order is:
0
the no. of outlier in list_price is:
0
the no. of outlier in standard_cost is:
195
the no. of outlier in product_first_sold_date is:
0


#### standard_cost coloumn 195 outliers.
*so we can replace the outliers by mean of standard cost of that particular product*

#### Checking the categorical columns , if there is different entry or values indiacating same category then change it as one category

In [20]:
transactions["online_order"].unique()

array([ 0.,  1., nan])

In [21]:
transactions["order_status"].unique()

array(['Approved', 'Cancelled'], dtype=object)

In [22]:
transactions["brand"].unique()

array(['Solex', 'Trek Bicycles', 'OHM Cycles', 'Norco Bicycles',
       'Giant Bicycles', 'WeareA2B', nan], dtype=object)

In [23]:
transactions["product_line"].unique()

array(['Standard', 'Road', 'Mountain', 'Touring', nan], dtype=object)

In [24]:
transactions["product_class"].unique()

array(['medium', 'low', 'high', nan], dtype=object)

In [25]:
transactions["product_size"].unique()

array(['medium', 'large', 'small', nan], dtype=object)

#### There is no such data that contradict the values of categorical columns.
*there is some null value as we saw earlier*

#### As per data given there is wrong data fetched in "product_first_sold_date" column
*column must be in date format*

In [26]:
transactions["product_first_sold_date"]=pd.TimedeltaIndex(transactions["product_first_sold_date"],unit='d')+dt.datetime(1900,1,1)
transactions["product_first_sold_date"]

0       2012-12-04
1       2014-03-05
2       1999-07-22
3       1998-12-18
4       2015-08-12
           ...    
19995   2003-07-23
19996   1997-05-12
19997   2010-08-22
19998   2004-08-19
19999   1999-06-25
Name: product_first_sold_date, Length: 20000, dtype: datetime64[ns]

#### So change the column value into proper format for proper analysis